# Documentación de Sentencias SQL: Análisis de Ventas de Cafetería

### Introducción

Este documento presenta la documentación de las sentencias SQL utilizadas para el análisis exploratorio y la preparación de datos de una tienda de café. A través de estas consultas, se limpiarán los datos, se transformarán las estructuras de las tablas y se extraerán métricas clave para entender el rendimiento del negocio.

### Sentencias SQL Utilizadas y su Acción

A continuación, se definen las principales sentencias y funciones de SQL empleadas en este análisis y la acción que realizan:

* `SELECT`: Recupera datos de una o más tablas en una base de datos.
* `FROM`: Especifica la(s) tabla(s) de la(s) cual(es) se van a recuperar los datos.
* `WHERE`: Filtra los registros para extraer solo aquellos que cumplan una condición específica.
* `GROUP BY`: Agrupa filas que tienen los mismos valores en columnas especificadas en filas de resumen.
* `ORDER BY`: Ordena el conjunto de resultados de una consulta en orden ascendente o descendente.
* `UPDATE`: Modifica los datos existentes en una tabla.
* `SET`: Se utiliza con `UPDATE` para especificar las columnas y los nuevos valores a modificar.
* `ALTER TABLE`: Modifica la estructura existente de una tabla (añadir, eliminar o modificar columnas, cambiar tipos de datos, etc.).
* `MODIFY COLUMN`: Se utiliza con `ALTER TABLE` para cambiar el tipo de dato de una columna.
* `CHANGE COLUMN`: Se utiliza con `ALTER TABLE` para renombrar una columna y/o cambiar su tipo de dato.
* `DESCRIBE`: Muestra la estructura de una tabla, incluyendo columnas, tipos de datos, claves, etc.
* `LIMIT`: Restringe el número de filas devueltas por una consulta.
* `CASE`: Permite realizar lógica condicional en una consulta (similar a una sentencia if/else).
* `AVG()`: Función de agregación que calcula el promedio de un conjunto de valores.
* `SUM()`: Función de agregación que calcula la suma total de un conjunto de valores.
* `COUNT()`: Función de agregación que cuenta el número de filas que cumplen una condición específica.
* `ROUND()`: Redondea un número a un número específico de decimales.
* `MONTH()`: Extrae el número del mes de una fecha.
* `DAYOFWEEK()`: Extrae el día de la semana de una fecha (el valor numérico depende del sistema, a menudo 1 es Domingo, 7 es Sábado).
* `HOUR()`: Extrae la hora de una hora o marca de tiempo.
* `STR_TO_DATE()`: Convierte una cadena de caracteres en un valor de fecha y/o hora.
* `REPLACE()`: Reemplaza todas las ocurrencias de una subcadena dentro de una cadena por otra subcadena.
* `CONCAT()`: Concatena dos o más cadenas de caracteres.
* `LAG()`: Función de ventana que accede a una fila anterior en algún número de desplazamiento dentro de la misma partición y devuelve el valor de una columna de esa fila anterior. Se utiliza comúnmente para comparar el valor de la fila actual con el de una fila anterior.
* `OVER()`: Se utiliza con funciones de ventana (`LAG`, `AVG` aquí usado como función de ventana) para definir la ventana o conjunto de filas sobre las que opera la función.

---


### Documentación por Bloque de Código SQL

A continuación, se presenta la documentación de cada bloque de código SQL ejecutado:

**Bloque 1: Exploración Inicial de la Tabla**

```sql
SELECT * FROM coffee_shop_sales

![](images/Select.png)

DESCRIBE coffee_shop_sales

Esta sentencia muestra la estructura de la tabla coffee_shop_sales. Proporciona información sobre el nombre de cada columna, su tipo de dato, si acepta valores nulos, claves, etc. Sirve para obtener información sobre la estructura de una tabla.

![](images/Describe.png)

- - - - - - - - - - - 


**Bloque 2: Limpieza y Transformación de Tipos de Datos (Fecha y Hora)**


UPDATE coffee_shop_sales
SET transaction_date = STR_TO_DATE(transaction_date, '%d-%m-%Y');

 - Esta consulta de actualización modifica la columna transaction_date. Utiliza la función STR_TO_DATE para convertir la cadena de texto existente en la columna transaction_date a un formato de fecha, asumiendo que el formato original es día-mes-año (ej. '31-12-2023'). 




ALTER TABLE coffee_shop_sales
MODIFY COLUMN transaction_date DATE;

 - Una vez que los valores de transaction_date han sido convertidos a un formato interpretable como fecha, esta sentencia modifica la estructura de la tabla para cambiar explícitamente el tipo de dato de la columna transaction_date a DATE. Esto asegura que la base de datos reconozca y maneje correctamente esta columna como fechas. 




UPDATE coffee_shop_sales
SET transaction_time = STR_TO_DATE(transaction_time, '%H:%i:%s');

 - Similar al proceso con la fecha, esta consulta actualiza la columna transaction_time, convirtiendo la cadena de texto de la hora a un formato de hora reconocido por la base de datos, asumiendo un formato hora:minuto:segundo (ej. '14:30:00'). 





ALTER TABLE coffee_shop_sales
MODIFY COLUMN transaction_time TIME;

 - Esta sentencia modifica la estructura de la tabla para cambiar el tipo de dato de la columna transaction_time a TIME, asegurando su correcto manejo como valores de tiempo. 




 ALTER TABLE coffee_shop_sales
CHANGE COLUMN `¿transaction_id` transaction_id INT;

 - Esta sentencia ALTER TABLE realiza dos acciones sobre una columna. Utiliza CHANGE COLUMN para renombrar la columna de ¿transaction_id a transaction_id (eliminando un posible carácter erróneo inicial) y simultáneamente cambia su tipo de dato a INT (entero).


- - - - - - - - - - - - -

**Bloque 3: Cálculo de Ventas Totales para un Mes Específico**


SELECT ROUND (SUM(unit_price* transaction_qty)) as Total_Sales
FROM coffee_shop_sales
WHERE MONTH(transaction_date) = 5 -- for month of (CM-May)

 - Esta consulta calcula las ventas totales para un mes específico (en este caso, Mayo, filtrando donde el número del mes es 5). Multiplica el unit_price por transaction_qty para cada fila, suma estos valores (SUM), y redondea el resultado (ROUND). El resultado se presenta con el alias Total_Sales.

![](images/Total%20ventas%20Por%20mes.png)

**Bloque 4: Cálculo de Ventas Totales y Crecimiento Mes a Mes (MoM)**


SELECT
    MONTH(transaction_date) AS month,

    ROUND(SUM(unit_price* transaction_qty)) AS total_sales,

    (SUM(unit_price* transaction_qty) - LAG(SUM(unit_price* transaction_qty), 1) OVER (ORDER BY MONTH(transaction_date))) / LAG

    (SUM(unit_price* transaction_qty), 1) OVER (ORDER BY MONTH(transaction_date)) * 100 AS mom_increase_percentage

FROM

    coffee_shop_sales

WHERE

    MONTH(transaction_date) IN (4, 5) -- for months of April and May

GROUP BY

    MONTH(transaction_date)

ORDER BY

    MONTH(transaction_date);

 - Esta consulta calcula las ventas totales para los meses de Abril y Mayo y determina el crecimiento porcentual mes a mes (MoM). 
Agrupa los datos por mes (GROUP BY MONTH(transaction_date)).
Calcula la suma de unit_price * transaction_qty para obtener las ventas totales por mes.
Utiliza la función de ventana LAG() para obtener el total de ventas del mes anterior (LAG(SUM(unit_price* transaction_qty), 1)).
La fórmula (Ventas Actual - Ventas Anterior) / Ventas Anterior * 100 calcula el porcentaje de crecimiento MoM.
Filtra los datos para incluir solo los meses 4 y 5 (WHERE MONTH(transaction_date) IN (4, 5)).
Ordena los resultados por número de mes.

![](images/incventas.png)


- - - - -- - - - - 

**Bloque 5: Cálculo del Total de Pedidos para un Mes Específico**


SELECT COUNT(transaction_id) as Total_Orders

FROM coffee_shop_sales

WHERE MONTH (transaction_date)= 5 -- for month of (CM-May)

- Esta consulta calcula el número total de pedidos (contando las filas por transaction_id) para un mes específico (Mayo, MONTH(transaction_date) = 5).

![](images/Total%20pedidos%20por%20mes.png)

**Cálculo del Total de Pedidos y Crecimiento Mes a Mes (MoM)**


SELECT
    MONTH(transaction_date) AS month,

    ROUND(COUNT (transaction_id)) AS total_orders,

    (COUNT(transaction_id) - LAG(COUNT (transaction_id), 1) OVER (ORDER BY MONTH(transaction_date))) / LAG(COUNT 

    (transaction_id), 1) OVER (ORDER BY MONTH(transaction_date)) * 100 AS mom_increase_percentage

FROM

    coffee_shop_sales

WHERE

    MONTH(transaction_date) IN (4, 5) -- for April and May

GROUP BY

    MONTH(transaction_date)

ORDER BY

    MONTH(transaction_date);

- Calcula el total de pedidos (COUNT(transaction_id)) para los meses de Abril y Mayo y determina el crecimiento porcentual de pedidos mes a mes (MoM).

![](images/inc%20y%20dif%20entre%20pedidos%20por%20mes.png)

- - - - - - - 

**Bloque 6: Calculo de Total Productos Vendidos** 

SELECT SUM(transaction_qty) as Total_Quantity_Sold

FROM coffee_shop_sales

WHERE MONTH(transaction_date) = 5 -- Para el mes de mayo

Esta consulta suma la cantidad total de artículos vendidos (transaction_qty) para un mes específico (Mayo).

![](images/Cantidad_de_prod_vendidos.png)

**Cálculo de la Cantidad Total Vendida y Crecimiento Mes a Mes (MoM)**


SELECT

    MONTH(transaction_date) AS month,

    ROUND(SUM(transaction_qty)) AS total_quantity_sold,

    (SUM(transaction_qty) - LAG(SUM(transaction_qty), 1) OVER (ORDER BY MONTH(transaction_date))) / LAG(SUM

    (transaction_qty), 1) OVER (ORDER BY MONTH(transaction_date)) * 100 AS mom_increase_percentage

FROM

    coffee_shop_sales

WHERE

    MONTH(transaction_date) IN (4, 5) -- for April and May

GROUP BY

    MONTH(transaction_date)

ORDER BY

    MONTH(transaction_date);

Esta consulta calcula la cantidad total de productos vendidos (SUM(transaction_qty)) para los meses de Abril y Mayo y determina el crecimiento porcentual de la cantidad vendida mes a mes (MoM)

![](images/Inc_y_dif_entre_cantidad_de_productos.png)

- - - - - - - - -- - - 

**Bloque 7: Cálculo de Métricas Clave para un Día Específico**

SELECT

    SUM(unit_price* transaction_qty) AS total_sales,

    SUM(transaction_qty) AS total_quantity_sold,

    COUNT(transaction_id) AS total_orders

FROM

    coffee_shop_sales

WHERE

    transaction_date = '2023-05-18'; -- Para 18 Mayo 2023

Esta consulta obtiene las métricas clave (ventas totales, cantidad total vendida y número total de pedidos) para un día específico ('2023-05-18'). 


![](images/Resultadosxdia.png)

**Cálculo y Formateo de Métricas Clave para un Día Específico (en Miles)**


SELECT

    CONCAT(ROUND(SUM(unit_price* transaction_qty) / 1000, 1), 'K') AS total_sales,

    CONCAT(ROUND (COUNT (transaction_id) / 1000, 1),'K') AS total_orders,

    CONCAT(ROUND (SUM(transaction_qty) / 1000, 1), 'K') AS total_quantity_sold

FROM

    coffee_shop_sales

WHERE

    transaction_date = '2023-05-18'; --For 18 May 2023

Esta consulta formatea los resultados en miles (dividiendo por 1000 y añadiendo 'K' al final) y redondea a un decimal utilizando ROUND y CONCAT. Presenta las ventas totales, el número total de pedidos y la cantidad total vendida para la fecha '2023-05-18'.

![](images/Ventasxdiaredondeado.png)

- - - - - -

**Bloque 8: Cálculo del Promedio de Ventas Diarias para un Mes**


SELECT AVG(total_sales) AS average_sales

FROM (

    SELECT

        SUM(unit_price* transaction_qty) AS total_sales

    FROM

        coffee_shop_sales

    WHERE

        MONTH(transaction_date) = 5 -- Filter for May

    GROUP BY

        transaction_date

) AS internal_query;

Esta consulta calcula el promedio de las ventas diarias para el mes de Mayo. 
Una subconsulta (aliased como internal_query) calcula las ventas totales para cada día de Mayo (SUM(...) GROUP BY transaction_date). 
La consulta externa calcula el promedio (AVG()) de los resultados de ventas diarias obtenidos en la subconsulta. 






![](images/Promedio.png)

**Bloque 9: Ventas Totales por Día del Mes**


SELECT

    DAY(transaction_date) AS day_of_month,

    ROUND(SUM(unit_price* transaction_qty), 1) AS total_sales

FROM

    coffee_shop_sales

WHERE

    MONTH(transaction_date) = 5

GROUP BY

    DAY(transaction_date)

ORDER BY

    DAY(transaction_date);

Esta consulta agrupa las transacciones por cada día del mes de Mayo y calcula la suma total de ventas para cada día. Los resultados se presentan ordenados por el día del mes.


![](images/Diadelmes1.png)     ![](images/Diadelmes2.png)

**Bloque 10: Categorización de Ventas Diarias (Por Encima/Debajo del Promedio)**

SELECT

    day_of_month,

    CASE

        WHEN total_sales > avg_sales THEN 'Above Average'

        WHEN total_sales < avg_sales THEN 'Below Average'

        ELSE 'Average'

    END AS sales_status,

    total_sales

FROM (

    SELECT

        DAY(transaction_date) AS day_of_month,

        SUM(unit_price* transaction_qty) AS total_sales,

        AVG(SUM(unit_price* transaction_qty)) OVER () AS avg_sales

    FROM

        coffee_shop_sales

    WHERE

        MONTH(transaction_date) = 5

    GROUP BY

        DAY(transaction_date)

) AS sales_data

ORDER BY

    day_of_month;

Esta consulta categoriza el rendimiento de ventas de cada día de Mayo como 'Above Average' (Por encima del promedio), 'Below Average' (Por debajo del promedio) o 'Average' (Promedio). 
Una subconsulta (aliased como sales_data) calcula las ventas totales por día y el promedio general de ventas diarias para todo el mes (utilizando AVG() como función de ventana con OVER ()). 
La consulta externa utiliza una sentencia CASE para comparar las ventas de cada día (total_sales) con el promedio (avg_sales) y asignar la categoría correspondiente. 
Los resultados se ordenan por día del mes.

![](images/Porencimadel.png)

- - - -- - - 

**Bloque 11: Ventas Totales por Tipo de Día (Fin de Semana vs. Día de Semana)**


SELECT

    CASE

        WHEN DAYOFWEEK(transaction_date) IN (1, 7) THEN 'FindeSemana'

        ELSE 'DiasdeSemana'

    END AS day_type,

    ROUND(SUM(unit_price* transaction_qty), 2) AS total_sales

FROM

    coffee_shop_sales

WHERE

    MONTH(transaction_date) = 5 -- Filter for May

GROUP BY

    CASE

        WHEN DAYOFWEEK (transaction_date) IN (1, 7) THEN 'FindeSemana'

        ELSE 'DiasdeSemana'

    END;

Esta consulta calcula las ventas totales para el mes de Mayo, diferenciando entre las ventas realizadas durante los fines de semana (DAYOFWEEK es 1 para Domingo, 7 para Sábado) y los días de semana. Agrupa los resultados según el tipo de día ('FindeSemana' o 'DiasdeSemana')

![](images/SemanaFIndesemana.png)

- - - - - 

**Bloque 12: Ventas Totales por Categoría de Producto**


SELECT

    product_category,

    ROUND(SUM(unit_price* transaction_qty), 1) as Total_Sales

FROM coffee_shop_sales

WHERE

    MONTH(transaction_date) = 5

GROUP BY product_category

ORDER BY SUM(unit_price* transaction_qty) DESC

Esta consulta calcula las ventas totales para cada categoría de producto en el mes de Mayo. Agrupa los resultados por product_category y los ordena de forma descendente según el total de ventas, mostrando las categorías más vendidas primero. 

![](images/Categorias-ventas.png)

- - - - - 

**Bloque 13: Ventas por Producto (Top 10)**


SELECT

    product_type,

    ROUND(SUM(unit_price* transaction_qty), 1) as Total_Sales

FROM coffee_shop_sales

WHERE

    MONTH(transaction_date) = 5

GROUP BY product_type

ORDER BY SUM(unit_price* transaction_qty) DESC

LIMIT 10

Similar al bloque anterior, pero esta consulta calcula las ventas totales para cada tipo de producto en el mes de Mayo. Agrupa por product_type, ordena por ventas descendentes y utiliza LIMIT 10 para mostrar solo los 10 productos más vendidos.

![](images/Top10.png)

- - - - - -  - - -

**Bloque 14: Ventas para un Día y Hora Específicos**


SELECT

    ROUND(SUM(unit_price* transaction_qty)) AS Total_Sales,

    SUM(transaction_qty) AS Total_Quantity,

    COUNT(*) AS Total_Orders

FROM

    coffee_shop_sales

WHERE

    DAYOFWEEK(transaction_date) = 3 -- Filtro para Martes

    AND HOUR(transaction_time) = 8 -- Filro para hora 8:00

    AND MONTH(transaction_date) = 5; -- Filtro para mes de Mayo

Esta consulta calcula las ventas totales, la cantidad total vendida y el número total de pedidos para un día de la semana (Martes) y una hora (HOUR = 8) específicos dentro del mes de Mayo. 

![](images/Ventasxhora.png)

- - - -- - - - 

**Bloque 15: Ventas Totales por Día de la Semana**

SELECT

    CASE

        WHEN DAYOFWEEK (transaction_date) = 2 THEN 'Monday'

        WHEN DAYOFWEEK(transaction_date) = 3 THEN 'Tuesday'

        WHEN DAYOFWEEK(transaction_date) = 4 THEN 'Wednesday'

        WHEN DAYOFWEEK (transaction_date) = 5 THEN 'Thursday'

        WHEN DAYOFWEEK (transaction_date) = 6 THEN 'Friday'

        WHEN DAYOFWEEK(transaction_date) = 7 THEN 'Saturday'

        ELSE 'Sunday'

    END AS Day_of_Week,

    ROUND(SUM(unit_price* transaction_qty)) AS Total_Sales

FROM

    coffee_shop_sales

WHERE

    MONTH(transaction_date) = 5

GROUP BY

    CASE

        WHEN DAYOFWEEK(transaction_date) = 2 THEN 'Monday'

        WHEN DAYOFWEEK(transaction_date) = 3 THEN 'Tuesday'

        WHEN DAYOFWEEK (transaction_date) = 4 THEN 'Wednesday'

        WHEN DAYOFWEEK (transaction_date) = 5 THEN 'Thursday'

        WHEN DAYOFWEEK (transaction_date) = 6 THEN 'Friday'

        WHEN DAYOFWEEK(transaction_date) = 7 THEN 'Saturday'

        ELSE 'Sunday'

    END;

Esta consulta calcula las ventas totales para cada día de la semana (Lunes a Domingo en ingles para seguir el lenguaje de la tabla) en el mes de Mayo. Utiliza una sentencia CASE para asignar un nombre legible a cada número de día de la semana y agrupa los resultados por este nombre del día. 

![](images/Ventas_por_dia.png)

- - - -- 

**Bloque 16: Ventas Totales por Hora del Día**


SELECT

    HOUR(transaction_time) AS Hour_of_Day,

    ROUND(SUM(unit_price* transaction_qty)) AS Total_Sales

FROM

    coffee_shop_sales

WHERE

    MONTH(transaction_date) = 5 -- Filter for May (month number 5)

GROUP BY

    HOUR(transaction_time)

ORDER BY

    HOUR(transaction_time);

Esta consulta calcula las ventas totales para cada hora del día (0 a 23) en el mes de Mayo. Agrupa los resultados por el número de la hora y los ordena en orden ascendente. 

![](images/Ventasporhoraenelmesdemayo.png)

**Se realizara un informe y graficas en Power BI para visualizar la información de los datos obtenidos de esta base de datos**